# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4360da9cd0>
├── 'a' --> tensor([[-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245]])
└── 'x' --> <FastTreeValue 0x7f4360da9ca0>
    └── 'c' --> tensor([[-0.3563, -1.0826,  0.4630, -0.0472],
                        [-0.0239,  1.3690, -1.6710, -0.9967],
                        [-0.1294,  0.2696, -0.0838,  0.5286]])

In [4]:
t.a

tensor([[-1.7661,  0.4394, -0.1520],
        [-0.5523,  0.3536, -1.4245]])

In [5]:
%timeit t.a

64.8 ns ± 0.0696 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4360da9cd0>
├── 'a' --> tensor([[-1.5974, -0.5976, -0.7977],
│                   [-0.1169,  0.2271,  0.5161]])
└── 'x' --> <FastTreeValue 0x7f4360da9ca0>
    └── 'c' --> tensor([[-0.3563, -1.0826,  0.4630, -0.0472],
                        [-0.0239,  1.3690, -1.6710, -0.9967],
                        [-0.1294,  0.2696, -0.0838,  0.5286]])

In [7]:
%timeit t.a = new_value

67.3 ns ± 0.0545 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245]]),
    x: Batch(
           c: tensor([[-0.3563, -1.0826,  0.4630, -0.0472],
                      [-0.0239,  1.3690, -1.6710, -0.9967],
                      [-0.1294,  0.2696, -0.0838,  0.5286]]),
       ),
)

In [10]:
b.a

tensor([[-1.7661,  0.4394, -0.1520],
        [-0.5523,  0.3536, -1.4245]])

In [11]:
%timeit b.a

57.9 ns ± 0.0807 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1311, -1.0728,  0.7786],
               [-0.0244,  0.6969,  0.3898]]),
    x: Batch(
           c: tensor([[-0.3563, -1.0826,  0.4630, -0.0472],
                      [-0.0239,  1.3690, -1.6710, -0.9967],
                      [-0.1294,  0.2696, -0.0838,  0.5286]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.0694 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 2.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 429 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 872 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f42ac835e20>
├── 'a' --> tensor([[[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]],
│           
│                   [[-1.7661,  0.4394, -0.1520],
│                    [-0.5523,  0.3536, -1.4245]]])
└── 'x' --> <FastTreeValue 0x7f42bc717670>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 82.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4360db0430>
├── 'a' --> tensor([[-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245],
│                   [-1.7661,  0.4394, -0.1520],
│                   [-0.5523,  0.3536, -1.4245]])
└── 'x' --> <FastTreeValue 0x7f4360db0250>
    └── 'c' --> tensor([[-0.3563, -1.0826,  0.4630, -0.0472],
                        [-0.0239,  1.3690, -1.6710, -0.9967],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 40.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.2 µs ± 73.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]],
       
               [[-1.7661,  0.4394, -0.1520],
                [-0.5523,  0.3536, -1.4245]]]),
    x: Batch(
           c: tensor([[[-0.3563, -1.0826,  0.4630, -0.0472],
                       [-0.0239,  1.3690, -1.6710, -0.9967],
                       [-0.1294,  0.2696, -0.0838,  0.5286]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245],
               [-1.7661,  0.4394, -0.1520],
               [-0.5523,  0.3536, -1.4245]]),
    x: Batch(
           c: tensor([[-0.3563, -1.0826,  0.4630, -0.0472],
                      [-0.0239,  1.3690, -1.6710, -0.9967],
                      [-0.1294,  0.2696, -0.0838,  0.5286],
                      [-0.3563, -1.0826,  0.4630, -0.0472],
                      [-0.0239,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 605 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
